In [1]:
from dotenv import load_dotenv
import os
from qiskit import *
from qiskit_ibm_provider import IBMProvider
from math import *
import qiskit


### Initial Setup for Qiskit, loading accounts and Providers

In [2]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.24.0', 'qiskit-aer': '0.12.0', 'qiskit-ignis': None, 'qiskit-ibmq-provider': '0.20.2', 'qiskit': '0.43.0', 'qiskit-nature': None, 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [3]:
load_dotenv()
IBM_KEY = os.getenv("API_KEY")

In [4]:
provider = IBMProvider()

### Code for initializing a qubit

In [5]:
# Initializing a single qubit to |0> and naming the qubit as 'q'
qr = QuantumRegister(1, "q")
qr

QuantumRegister(1, 'q')

In [6]:
qr[0]

Qubit(QuantumRegister(1, 'q'), 0)

In [7]:
# Initializing a qubit to a desired state
desired_state = [1/sqrt(2), 1/sqrt(2)]
qr = QuantumRegister(1, "q")
qc = QuantumCircuit(qr)
qc.initialize(desired_state, qr[0])
qc.draw()

┌─────────────────────────────┐
q: ┤ Initialize(0.70711,0.70711) ├
   └─────────────────────────────┘

#### Some basic gates are as follows

$$X = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}$$
$$Y = \begin{bmatrix} 0 & -i \\ i & 0 \end{bmatrix}$$
$$Z = \begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix}$$
$$H = \frac{1}{\sqrt2}\begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix}$$
$$S = \begin{bmatrix} 1 & 0 \\ 0 & i \end{bmatrix}$$
$$T = \begin{bmatrix} 1 & 0 \\ 0 & e^{i\pi /4} \end{bmatrix}$$

##### Some useful states that can also be used as basis since they are orthogonal are:

$$| + \rangle = \frac{1}{\sqrt2}(|0\rangle + |1\rangle)$$
$$| - \rangle = \frac{1}{\sqrt2}(|0\rangle - |1\rangle)$$

### A quantum circuit implementing gates H, Z, X

This circuit uses H, Z, and X gates in that order to obtain the state $\frac{1}{\sqrt2}(|1\rangle - |0\rangle)$ from $|0\rangle$

First, by default, qr is initialized to $|0\rangle$, and on application of H, we obtained $\frac{1}{\sqrt2}(|0\rangle + |1\rangle)$  
Then application of Z gate makes qr evolve to $\frac{1}{\sqrt2}(|0\rangle - |1\rangle)$  
Finally, X acts similar to the classical NOT gate and qr finally evolves to $\frac{1}{\sqrt2}(|1\rangle - |0\rangle)$

In [8]:
qr = QuantumRegister(1)
qc = QuantumCircuit(qr)
qc.h(qr)
qc.z(qr)
qc.x(qr)
qc.draw()

┌───┐┌───┐┌───┐
q0: ┤ H ├┤ Z ├┤ X ├
    └───┘└───┘└───┘

### Generalization of single qubit gates

$$U_3(\theta, \phi, \lambda) = \begin{bmatrix} \cos \frac{\theta}{2} & -e^{i\lambda}\sin \frac{\theta}{2} \\[6pt] e^{i\phi}\sin \frac{\theta}{2} & e^{i(\phi + \lambda)\cos \frac{\theta}{2}} \end{bmatrix}$$  
We can express X gate as $U_3(\pi, 0, \pi)$, H gate as $U_3(\frac{\pi}{2}, 0, \pi)$

#### Other common parametric gates $U_1$ and $U_2$ are as follows:
$$U_1(\lambda) = \begin{bmatrix} 1 & 0 \\[6pt] 0 & e^{i\lambda} \end{bmatrix} = U_3(0, 0, \lambda)$$

$$U_2(\phi, \lambda) = \frac{1}{\sqrt2}\begin{bmatrix} 1 & -e^{i\lambda} \\[6pt] e^{i\phi} & e^{i(\phi + \lambda)} \end{bmatrix} = U_3(\frac{\pi}{2}, \phi, \lambda)$$


### Evolving to the same state as above using U1, U2, U3 gates

In [9]:
qr = QuantumRegister(1)
qc = QuantumCircuit(qr)
qc.u(pi/2, 0, pi, qr) #u2 gate
qc.u(0, 0, pi, qr) #u1 gate
qc.u(pi, 0, pi, qr) #u3 gate

In [10]:
backend = Aer.get_backend('statevector_simulator')
qjob = execute(qc, backend)
out_vector = qjob.result().get_statevector()
print(out_vector)

Statevector([-0.70710678+8.65956056e-17j,  0.70710678+0.00000000e+00j],
            dims=(2,))


### Single Qubit Measurement

We flip the qubit qr and measure it 100 times. Since initial state of qr was $|0\rangle$, flipping it makes its state $|1\rangle$, so measuring it 100 times should give us the state $|1\rangle$ 100 times.  
This measurement is performed in computational basis, and qubit will collapse to one of $|0\rangle$ or $|1\rangle$ on measurement

In [11]:
# Creating Quantum Register, Classical Register, and Quantum Circuit
qr = QuantumRegister(1)
cr = ClassicalRegister(1)  # Helps in storing the output of the qubit measurement
qc = QuantumCircuit(qr, cr)

# Flipping qubit qr and measuring it
qc.x(qr)
qc.measure(qr, cr) # Measure the qubit qr and store the ouput in cr

# Executing on backend
backend = Aer.get_backend('qasm_simulator')
qjob = execute(qc, backend, shots=100) # shots specifies the number of times the circuit (including measurement) will be repeated
# Resulting measurement statistic will be stored in result object
# This mimics measuring 100 qubits in computational basis
measurement = qjob.result().get_counts()
print(measurement)

{'1': 100}


#### Measuring in Hadamard Basis

We can measure the state of the qubit in any orthogonal basis. If we want to know whether the output is $|+\rangle$ or $|-\rangle$, then we have to measure in the Hadamard Basis: $\{ |+\rangle, |-\rangle \}$  
Any state $|\phi \rangle = \alpha |0\rangle + \beta |1\rangle$ can be represented in Hadamard basis using the following:
$$|\phi \rangle = \frac{(\alpha + \beta)}{\sqrt2} |+\rangle + \frac{(\alpha - \beta)}{\sqrt2} |-\rangle$$

Now, if we apply $H^{\dagger}$ on $|\phi\rangle$, we have:
$$H^{\dagger} |\phi \rangle = \frac{(\alpha + \beta)}{\sqrt2} |0\rangle + \frac{(\alpha - \beta)}{\sqrt2} |1\rangle$$
i.e., $H^{\dagger}$ maps $|+\rangle$ to $|0\rangle$ and $|-\rangle$ to $|1\rangle$

So, measuring $H^\dagger |\phi \rangle$ in computational basis is the same as measuring $|\phi \rangle$ in Hadamard Basis  
This is called Transformation of Basis

Note: $H = H^\dagger$

In [17]:
qr = QuantumRegister(1)
cr = ClassicalRegister(1)
qc = QuantumCircuit(qr, cr)

qc.x(qr)
# Now we need to measure whether output is one of the Hadamard Basis
# So first apply H^dagger = H on qr to transform to computational basis
# In output, 0 corresponds to + and 1 corresponds to -

qc.h(qr)
qc.measure(qr, cr)

backend = Aer.get_backend('qasm_simulator')
qjob = execute(qc, backend) # Default shots = 1024
counts = qjob.result().get_counts()
print(counts)


{'1': 488, '0': 536}


Slight deviation due to randomness, otherwise should have been prob = $\frac{1}{2}$ for each exactly  
Note: In output, $|0\rangle$ corresponds to $|+\rangle$ and $|1\rangle$ corresponds to $|-\rangle$